In [44]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("worker_module").getOrCreate()

from pyspark.sql import functions as F

Impute the null continent name with mode of continent name 
Which continent has the highest number of Countries?
Top 5 countries for each continent in terms of wine servings
Top country for each continent in terms of alcohol servings
Find the most popular wine color for continent - EU and AS

In [27]:
#READ the data from CSV file.

lms_data_1 = spark.read.csv("../Downloads/1535617931_LMS_DATA_1.csv",header=True,inferSchema=True)

lms_data_2 = spark.read.csv("../Downloads/1535617956_LMS_DATA_2.csv",header=True,inferSchema=True)

lms_data_1.show(5)

lms_data_2.show(5)

+------------+-----------------------+-------------------------+-----------------------+-------------------------------+--------------+
|Country_Name|Number_of_Beer_Servings|Number_of_Spirit_Servings|Number_of_Wine_servings|Pure_alcohol_Consumption_litres|Continent_Name|
+------------+-----------------------+-------------------------+-----------------------+-------------------------------+--------------+
| Afghanistan|                      0|                        0|                      0|                            0.0|            AS|
|     Albania|                     89|                      132|                     54|                            4.9|            EU|
|     Algeria|                     25|                        0|                     14|                            0.7|            AF|
|     Andorra|                    245|                      138|                    312|                           12.4|            EU|
|      Angola|                    217|          

In [62]:
#JOIN the two tables (Left join)

data = lms_data_1.join(lms_data_2,on=(lms_data_1.Country_Name == lms_data_2.Country_Name) & (lms_data_1.Continent_Name == lms_data_2.Continent_Name),how='left').select(lms_data_1['*'],lms_data_2['Wine color'])

In [63]:
data.show(5)

+------------+-----------------------+-------------------------+-----------------------+-------------------------------+--------------+----------+
|Country_Name|Number_of_Beer_Servings|Number_of_Spirit_Servings|Number_of_Wine_servings|Pure_alcohol_Consumption_litres|Continent_Name|Wine color|
+------------+-----------------------+-------------------------+-----------------------+-------------------------------+--------------+----------+
| Afghanistan|                      0|                        0|                      0|                            0.0|            AS|       Red|
|     Albania|                     89|                      132|                     54|                            4.9|            EU|     Brown|
|     Algeria|                     25|                        0|                     14|                            0.7|            AF|Blistering|
|     Andorra|                    245|                      138|                    312|                           12.

#### Impute the null continent name with mode of continent name 


In [57]:
# Get the mode of continent

continent_mode =  data.groupby("Continent_Name").count().orderBy("count", ascending=False).first()[0]

In [76]:
data_1 = data.fillna(continent_mode,subset=['Continent_Name'])

In [77]:
data_1.show()

+-----------------+-----------------------+-------------------------+-----------------------+-------------------------------+--------------+----------+
|     Country_Name|Number_of_Beer_Servings|Number_of_Spirit_Servings|Number_of_Wine_servings|Pure_alcohol_Consumption_litres|Continent_Name|Wine color|
+-----------------+-----------------------+-------------------------+-----------------------+-------------------------------+--------------+----------+
|      Afghanistan|                      0|                        0|                      0|                            0.0|            AS|       Red|
|          Albania|                     89|                      132|                     54|                            4.9|            EU|     Brown|
|          Algeria|                     25|                        0|                     14|                            0.7|            AF|Blistering|
|          Andorra|                    245|                      138|                   

#### Which continent has the highest number of Countries?


In [85]:
#Creating temp df in order to not to consider multiple records.
temp = data.groupBy('Continent_Name',"Country_Name").count()

In [96]:
count = temp.groupBy("Continent_Name").count().orderBy("count", ascending=False).first()

In [99]:
print("The",count[0],"continet has",count[1], "number of countries which is highest.")

The AF continet has 53 number of countries which is highest.


#### Top 5 countries for each continent in terms of wine servings


In [102]:
from pyspark.sql.window import Window

In [103]:
win = Window.partitionBy("Continent_Name").orderBy(desc('Number_of_Wine_servings'))

In [117]:
data_1.select('*', rank().over(win).alias('rank')) .filter(col('rank') <= 5).select(["Continent_Name","Country_Name",'rank',"Number_of_Wine_servings"]).show(30)

+--------------+-------------------+----+-----------------------+
|Continent_Name|       Country_Name|rank|Number_of_Wine_servings|
+--------------+-------------------+----+-----------------------+
|            SA|          Argentina|   1|                    221|
|            SA|            Uruguay|   2|                    220|
|            SA|              Chile|   3|                    172|
|            SA|           Paraguay|   4|                     74|
|            SA|               Peru|   5|                     21|
|            AS|               Laos|   1|                    123|
|            AS| Russian Federation|   2|                     73|
|            AS|       Turkmenistan|   3|                     32|
|            AS|            Lebanon|   4|                     31|
|            AS|              Japan|   5|                     16|
|            AS|              Syria|   5|                     16|
|            OC|          Australia|   1|                    212|
|         

#### Top country for each continent in terms of alcohol servings


In [119]:
win_1 = Window.partitionBy("Continent_Name").orderBy(desc('Pure_alcohol_Consumption_litres'))

In [124]:
data_1.select('*', rank().over(win_1).alias('rank')) .filter(col('rank') <=1).select(["Continent_Name","Country_Name",'rank',"Pure_alcohol_Consumption_litres"]).show(30)

+--------------+------------------+----+-------------------------------+
|Continent_Name|      Country_Name|rank|Pure_alcohol_Consumption_litres|
+--------------+------------------+----+-------------------------------+
|            SA|         Argentina|   1|                            8.3|
|            AS|Russian Federation|   1|                           11.5|
|            OC|         Australia|   1|                           10.4|
|            EU|           Belarus|   1|                           14.4|
|            AF|           Grenada|   1|                           11.9|
+--------------+------------------+----+-------------------------------+



##### Find the most popular wine color for continent - EU and AS

In [141]:
data.groupBy('Continent_Name',"Wine color").count().filter("Continent_Name in ('EU','AS')").orderBy(desc("count")).show(2)

+--------------+----------+-----+
|Continent_Name|Wine color|count|
+--------------+----------+-----+
|            EU|     Brown|   15|
|            AS|Blistering|   14|
+--------------+----------+-----+
only showing top 2 rows

